## Find out the users that post the most in a set of subreddits

edit the `sub_names` var to add/remove subreddits from the calculation

In [5]:
import praw
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

##### vars #####
sub_names = ['politics', 'all', 'news', 'interestingasfuck', 'whitePeopleTwitter', 'worldnews']
################

reddit = praw.Reddit(
    client_id=os.getenv('client_id'),
    client_secret=os.getenv('client_secret'),
    user_agent="submissions comments",
    username=os.getenv('username'),
)

def serialize_reddit(s): 
    return {
        "author": s.author,
        "created_utc": s.created_utc,
    }
def flatten(l):
    return [item for sublist in l for item in sublist]


submissions = list(flatten(reddit.subreddit(s).top(time_filter='month',limit=200) for s in sub_names))
df = pd.DataFrame([serialize_reddit(s) for s in submissions])
df['created_utc'] = pd.to_datetime(df.created_utc.values, unit='s', utc=True)

print(sub_names)

ndf = df.query('author != "None"')
print('Total submissions:', len(ndf))

print('Number of unique authors:', ndf.author.nunique())

csum = ndf.author.value_counts(1).cumsum()

bins = pd.cut(csum, [0., 0.25, 0.5, 0.75, 1.], labels=['25%','50%','75%','100%'])

print('\nQuartiles:')
print(csum.groupby(bins).size())

print('\nTop 5 authors:')
topa = pd.concat([ndf.author.value_counts().head(), ndf.author.value_counts(1).head()*100],axis=1)
topa.index.name = 'Author'
topa.columns = ('Count', 'Percent')
print(topa)
print('\n======\n')

wg50x2
['politics', 'all', 'news', 'interestingasfuck', 'whitePeopleTwitter', 'worldnews']
Total submissions: 1200
Number of unique authors: 806

Quartiles:
author
25%      52
50%     155
75%     299
100%    300
Name: author, dtype: int64

Top 5 authors:
                      Count   Percent
Author                               
DaFunkJunkie             29  2.420701
Aztery                   13  1.085142
urmomsuckedmeoff         11  0.918197
GroundbreakingSet187     10  0.834725
gymgirl2018               9  0.751252


